In [ ]:
from __future__ import division
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import h5py
import os
import six
from six.moves import range
import itertools
import matplotlib.pyplot as plt
import PIL
from tqdm import tqdm
from PIL import ImageOps
from PIL import ImageEnhance
import matplotlib.font_manager
from PIL import ImageDraw, ImageFont, ImageFilter
import tempfile

from deep_car.data import data_generator, discretize, continuous, augment_img, \
    augment_batch, batch_to_numpy, \
    get_steering_delta, get_steering_hist
from deep_car.model import get_model, discretize_mixture, log_p_x, log_p_x_pdf

In [ ]:
h5py.version.version

In [ ]:
data_dir = '../data'
model_dir = '../data/model'
model_name = 'steering_mixture_prob_exp'
tmp_dir = '../tmp'

os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
crop_size = (64, 48)

h5_train = h5py.File(os.path.join(data_dir, 'train.hdf5'))
h5_test = h5py.File(os.path.join(data_dir, 'test.hdf5'))


delta_discretize_min = -36 - 4.5
delta_discretize_max = 36 + 4.5
delta_discretize_buckets = 9

In [ ]:
(delta_discretize_max - delta_discretize_min) / 9

In [ ]:
print("{:10}| {:10}| {:30}".format("name", "dtype", "shape"))
print("-" * 40)
for name, dset in h5_train.items():
    print("{:10}| {:10}| {:30}".format(name, str(dset.dtype), str(dset.shape)))

In [ ]:
          
batch = next(data_generator(h5_train))
for name, arr in sorted(batch.items()):
    print("{:<17} | {:} ".format(name, arr.shape))

In [ ]:
batch['image'].min(), batch['image'].max(), 

In [ ]:
fig, axes = plt.subplots(3, 10, figsize=(15, 4))
for i, ax in enumerate(axes.flat):
    ax.set_title("{:.1f}".format(float(180 / np.pi * batch['steering_delta_00'][i])))
    ax.imshow(batch['image'][i], cmap='gray')
    ax.grid('off')
    ax.set_xticks([])
    ax.set_yticks([])

#### number of seconds per epoch

In [ ]:
#%%timeit -n 3

#for batch in data_generator(n_epoch=1):
#    pass

In [ ]:
deltas = {key: [] for key in batch.keys() if 'delta' in key}

for batch in data_generator(h5_train, n_epoch=1):
    for key, arr in batch.items():
        if 'delta' in key:
            deltas[key].append(arr)
deltas = {k: np.concatenate(arrs).flatten() for k, arrs in deltas.items()}

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 6))

for ax, (key, delta) in zip(axes.flatten(), sorted(deltas.items())):
    ax.hist(np.clip(180 / np.pi * delta, -45, 45), bins=30)
    ax.set_title(key)

In [ ]:
crop_size

In [ ]:
batch = next(data_generator(h5_train))

In [ ]:
images = [PIL.Image.fromarray(x) for x in batch["image"]]
img = images[0]
fig, axes = plt.subplots(4, 12, figsize=(20, 5))

for ax in axes[:1].flat:
    ax.imshow(np.array(img), cmap='gray', vmin=0, vmax=255)
    ax.set_xticks([])
    ax.set_yticks([])
for ax in axes[1:].flat:
    ax.imshow(np.array(augment_img(img)), cmap='gray', vmin=0, vmax=255)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
batch_aug = augment_batch(batch)

fig, axes = plt.subplots(3, 12, figsize=(15, 4))

for ax, img, steering in zip(axes.flat, batch_aug['image'], batch_aug['steering_delta_00']):
    ax.imshow((img), cmap='gray')
    ax.set_title("{:}".format(int(180*float(steering) / np.pi)))
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
get_steering_hist(batch).shape

In [ ]:
X, s_hist, y_delta, y_abs = batch_to_numpy(batch_aug)
print("data: {}, y_delta: {}, y_abs: {}".format(X.shape, y_delta.shape, y_abs.shape))
print("data max: {}, data min: {}".format(X.max(), X.min()))
print("y_delta max: {}, y_delta min: {}".format(y_delta.max(), y_delta.min()))
print("y_abs max: {}, y_abs min: {}".format(y_abs.max(), y_abs.min()))

In [ ]:
disc = discretize(np.linspace(delta_discretize_min, delta_discretize_max, num=90),
                  delta_discretize_buckets, min=delta_discretize_min, max=delta_discretize_max)
cont = continuous(disc, delta_discretize_buckets, min=delta_discretize_min, max=delta_discretize_max)

fig, ax = plt.subplots(1, 2, figsize=(8, 3))
ax[0].hist(disc, bins=9)
_ = ax[1].hist(cont, bins=20)

In [ ]:
list(batch.keys())

In [ ]:
n_mixtures = 6
y_delta_buckets = 9
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

len_steering_hist = get_steering_hist(batch).shape[-1]
len_steering_delta = get_steering_delta(batch).shape[-1]


x = tf.placeholder(tf.float32, shape=[None, crop_size[1], crop_size[0], 1], name='image')
steering_hist = tf.placeholder(tf.float32, shape=[None, len_steering_hist], name='steering_hist')

y_abs_true, y_delta_true, opt_op, \
    y_abs_discr_prob, y_abs_loss, y_delta_prob, y_delta_loss = \
        get_model(x, steering_hist, len_steering_delta, y_delta_buckets)

init_op = tf.global_variables_initializer()
sess.run(init_op)

history = {'y_delta': [], 'y_abs': []}

In [ ]:
probs, = sess.run([y_abs_discr_prob], feed_dict={x: X, steering_hist: s_hist})
print(probs.shape)
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(np.arange(180) - 90, np.exp(probs[1]))

In [ ]:
X, s_hist, y_delta_, y_abs_ = batch_to_numpy(augment_batch(batch))
dropout = np.random.binomial(1, 0.75, s_hist.shape)
s_hist = dropout * s_hist
dropout = np.random.binomial(1, 0.85, (s_hist.shape[0], 1))
s_hist = np.tile(dropout, (1, 4)) * s_hist

In [ ]:
s_hist

In [ ]:
tqdm_gen = tqdm(data_generator(h5_train, batch_size=256, n_epoch=1))
running_loss = 'init'
for batch in tqdm_gen:
    X, s_hist, y_delta_, y_abs_ = batch_to_numpy(augment_batch(batch))
    # y += np.random.normal(0, 0.005 * np.pi, y.shape)
    # y = np.clip(y, -np.pi / 2, np.pi / 2)
    dropout = np.random.binomial(1, 0.75, s_hist.shape)
    s_hist = dropout * s_hist
    dropout = np.random.binomial(1, 0.85, (s_hist.shape[0], 1))
    s_hist = np.tile(dropout, (1, 4)) * s_hist
    
    y_delta_loss_, y_abs_loss_, _ = sess.run([y_delta_loss, y_abs_loss, opt_op], feed_dict={
        x:X, steering_hist: s_hist,
        y_abs_true: y_abs_, y_delta_true: y_delta_})
    
    
    history['y_delta'].append(np.mean(y_delta_loss_))
    history['y_abs'].append(np.mean(y_abs_loss_))
    batch_loss = np.mean(y_delta_loss_) + np.mean(y_abs_loss_)
    if running_loss == 'init':
        running_loss = batch_loss
    else:
        running_loss = 0.9*running_loss + 0.1*batch_loss
    tqdm_gen.set_description('loss: {:.02f}'.format(running_loss))
    
    
saver = tf.train.Saver()
save_path = saver.save(sess, os.path.join(model_dir, model_name + ".ckpt"))

In [ ]:
saver = tf.train.Saver()
save_path = saver.save(sess, os.path.join(model_dir, model_name + ".ckpt"))

In [ ]:
plt.plot(history['y_delta'])
for name in history.keys():
    plt.plot(history[name], label=name)
plt.legend()

In [ ]:
probs, y_delta_prob_ = sess.run([y_abs_discr_prob, y_delta_prob], feed_dict={x: X, steering_hist: s_hist})
print(probs.shape)
fig, axes = plt.subplots(nrows=4, figsize=(15, 10))
for i, ax in enumerate(axes):
    ax.bar(np.arange(180) - 90, np.exp(probs[i]))

In [ ]:
rows = 10
cols = y_delta_prob_.shape[1]
bins = y_delta_prob_.shape[2]
fig, axes = plt.subplots(rows, cols+2, figsize=(2*cols, rows))

for i in range(rows):
    for j in range(cols+2):
        ax = axes[i, j]
        if j == 0:
            ax.imshow(X[i, :, :, 0], cmap='gray')
            continue
        if j == 1:
            ax.bar(np.arange(180) - 90, np.exp(probs[i]))
            continue
            
        j = j - 2
        ax.bar(np.arange(bins), y_delta_prob_[i, j])
        ax.set_ylim(0, 1)

for ax in axes.flat:
    ax.set_yticks([])
    ax.set_xticks([])

In [ ]:
def image_draw_info(img, steering_true, steering_pred, font=None):
    scale = 30
    pred_max = max(steering_pred)
    steering_pred = np.array([xi/pred_max * scale for xi in steering_pred])
    if font is None:
        fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
        mono_fonts = [f for f in fonts if "mono" in f.lower() and 'bold' in f.lower()]
        mono_font = mono_fonts[0]
        font = ImageFont.truetype(mono_font, 18)
    
    
    img = img.resize((640,480))
    img= img.convert("RGBA")
    draw = ImageDraw.Draw(img)

    
    angle = steering_true + np.pi/2
    x = 1/2 * img.size[0]
    y = 90
    x_start = x - 200
    
    legend_color = "#ff0000"
    pred_color = "#0099ffaa"
    draw.text((10,10), "real:{}".format(steering_true), fill="#00ff00ff", font=font)
    draw.text(
        (img.size[0] - 300,10), 
        "best predict:{}-> {:.2f}".format(np.argmax(steering_pred) - 90,max(steering_pred)), 
        fill=pred_color,
        font=font
    )
    
    draw.text((x_start-5, y+10), "-90", fill=legend_color, font=font)
    draw.text((x-5, y+10), "0", fill=legend_color, font=font)
    draw.text((x-5+ 180, y+10), "90", fill=legend_color, font=font)
    for i,prob in enumerate(steering_pred):
        draw.line([x+2*(i-90), y, x+2*(i-90), y-prob], fill=pred_color, width=2)
    
    draw.line([x+2*steering_true, y, x+2*steering_true, y - scale], fill="#00ff00ff", width=2)                       
    del draw
    
    return img




In [ ]:
data_stream = data_generator(batch_size=100, n_epoch=1, shuffle=False, train=False)
_ = next(data_stream)

In [ ]:
img_fnames = []
video_dir = tempfile.mkdtemp(dir=tmp_dir)
print(video_dir)
idx = 0
for batch in tqdm(data_stream):
    X, s_hist, y_delta_, y_abs_ = batch_to_numpy(batch)
    X = X[:, 6:-6, 8:-8, :]
    probs, y_delta_prob_ = sess.run([y_abs_discr_prob, y_delta_prob], 
                                    feed_dict={x: X, steering_hist: s_hist})
    for i in range(len(X)):
        img = PIL.Image.fromarray((255*(X[i, :, :, 0]/2 + 0.5)).astype(np.uint8), 'L')
        img = image_draw_info(img, y_abs_[i, 0]/np.pi * 180, np.exp(probs[i]))
        img_fname = os.path.join(video_dir, "{:06d}.png".format(idx))
        img.save(img_fname)
        idx += 1
    
    if idx > 500:
        break

In [ ]:
def save_image(img, i):
    img_fname = os.path.join(tmp_dir, "{:06d}.png".format(i))
    img.save(img_fname)
    
try:
    for i,test_pred,test_real, img in zip(range(len(predictions)), predictions, h5_file['steering'][:], h5_file['image'][:]):
        img = PIL.Image.fromarray(img)
        image_draw_info(img_scale, test_real, test_pred)
        save_image(img_scale,i)
        
    video = ImageSequenceClip(tmp_dir, fps=24, with_mask=False)
    video.write_videofile("test_set_evaluation.webm", ffmpeg_params=['-b:v', '0', '-crf', '20'])
finally:        
    shutil.rmtree(tmp_dir)